# FMCG Promotion Copy Re-use Recommender - Exploratory Data Analysis

This notebook provides a comprehensive exploratory data analysis of the FMCG promotion and SKU datasets to understand the data structure, patterns, and insights for building a promotion copy re-use recommendation system.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## 1. Data Loading and Overview

In [ ]:
# Load datasets
sku_df = pd.read_csv('data/sku_master.csv')
promos_df = pd.read_csv('data/promos.csv')

print("=== SKU Master Dataset ===")
print(f"Shape: {sku_df.shape}")
print(f"Columns: {list(sku_df.columns)}")
print("\nFirst 5 rows:")
print(sku_df.head())

print("\n=== Promotions Dataset ===")
print(f"Shape: {promos_df.shape}")
print(f"Columns: {list(promos_df.columns)}")
print("\nFirst 5 rows:")
print(promos_df.head())

## 2. Data Quality Assessment

In [ ]:
def assess_data_quality(df, dataset_name):
    print(f"\n=== {dataset_name} Data Quality Assessment ===")
    
    # Basic info
    print(f"Dataset shape: {df.shape}")
    print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024:.2f} KB")
    
    # Missing values
    missing = df.isnull().sum()
    missing_pct = (missing / len(df)) * 100
    missing_df = pd.DataFrame({
        'Missing Count': missing,
        'Missing %': missing_pct
    })
    print("\nMissing Values:")
    print(missing_df[missing_df['Missing Count'] > 0])
    
    # Data types
    print("\nData Types:")
    print(df.dtypes)
    
    # Duplicates
    duplicates = df.duplicated().sum()
    print(f"\nDuplicate rows: {duplicates} ({duplicates/len(df)*100:.2f}%)")
    
    return missing_df

sku_quality = assess_data_quality(sku_df, "SKU Master")
promos_quality = assess_data_quality(promos_df, "Promotions")

## 3. SKU Master Analysis

In [ ]:
# SKU distribution by brand and category
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Brand distribution
brand_counts = sku_df['brand'].value_counts()
axes[0].pie(brand_counts.values, labels=brand_counts.index, autopct='%1.1f%%', startangle=90)
axes[0].set_title('SKU Distribution by Brand')

# Category distribution
category_counts = sku_df['category'].value_counts()
axes[1].pie(category_counts.values, labels=category_counts.index, autopct='%1.1f%%', startangle=90)
axes[1].set_title('SKU Distribution by Category')

plt.tight_layout()
plt.show()

print("Brand Distribution:")
print(brand_counts)
print("\nCategory Distribution:")
print(category_counts)

In [ ]:
# Pack size analysis
print("Pack Size Analysis:")
print(sku_df['pack_size'].value_counts().sort_index())

# Extract numeric pack sizes for analysis
sku_df['pack_size_numeric'] = sku_df['pack_size'].str.extract('(\d+)').astype(float)

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(sku_df['pack_size_numeric'].dropna(), bins=20, edgecolor='black', alpha=0.7)
plt.title('Distribution of Pack Sizes (Numeric)')
plt.xlabel('Pack Size (g/ml)')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
pack_size_counts = sku_df['pack_size'].value_counts()
plt.bar(range(len(pack_size_counts)), pack_size_counts.values)
plt.title('Pack Size Distribution')
plt.xlabel('Pack Size')
plt.ylabel('Count')
plt.xticks(range(len(pack_size_counts)), pack_size_counts.index, rotation=45)

plt.tight_layout()
plt.show()

## 4. Promotions Analysis

In [ ]:
# Promotion distribution analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Touchpoint distribution
touchpoint_counts = promos_df['touchpoint'].value_counts()
axes[0,0].pie(touchpoint_counts.values, labels=touchpoint_counts.index, autopct='%1.1f%%')
axes[0,0].set_title('Promotion Distribution by Touchpoint')

# Category distribution
promo_category_counts = promos_df['category'].value_counts()
axes[0,1].pie(promo_category_counts.values, labels=promo_category_counts.index, autopct='%1.1f%%')
axes[0,1].set_title('Promotion Distribution by Category')

# Period distribution
period_counts = promos_df['period'].value_counts().sort_index()
axes[1,0].bar(period_counts.index, period_counts.values)
axes[1,0].set_title('Promotion Distribution by Period')
axes[1,0].set_xlabel('Period')
axes[1,0].set_ylabel('Count')
axes[1,0].tick_params(axis='x', rotation=45)

# Brand distribution in promotions
promo_brand_counts = promos_df['brand'].value_counts()
axes[1,1].bar(promo_brand_counts.index, promo_brand_counts.values)
axes[1,1].set_title('Promotion Distribution by Brand')
axes[1,1].set_xlabel('Brand')
axes[1,1].set_ylabel('Count')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("Promotion Distribution Summary:")
print(f"Touchpoints: {touchpoint_counts.to_dict()}")
print(f"Categories: {promo_category_counts.to_dict()}")
print(f"Periods: {period_counts.to_dict()}")
print(f"Brands: {promo_brand_counts.to_dict()}")

## 5. KPI Performance Analysis

In [ ]:
# KPI analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# KPI Lift distribution
axes[0,0].hist(promos_df['kpi_lift'], bins=15, edgecolor='black', alpha=0.7)
axes[0,0].set_title('Distribution of KPI Lift')
axes[0,0].set_xlabel('KPI Lift')
axes[0,0].set_ylabel('Frequency')
axes[0,0].axvline(promos_df['kpi_lift'].mean(), color='red', linestyle='--', 
                  label=f'Mean: {promos_df["kpi_lift"].mean():.3f}')
axes[0,0].legend()

# KPI ROI distribution
axes[0,1].hist(promos_df['kpi_roi'], bins=15, edgecolor='black', alpha=0.7)
axes[0,1].set_title('Distribution of KPI ROI')
axes[0,1].set_xlabel('KPI ROI')
axes[0,1].set_ylabel('Frequency')
axes[0,1].axvline(promos_df['kpi_roi'].mean(), color='red', linestyle='--', 
                  label=f'Mean: {promos_df["kpi_roi"].mean():.3f}')
axes[0,1].legend()

# KPI Lift by Touchpoint
touchpoint_lift = promos_df.groupby('touchpoint')['kpi_lift'].mean().sort_values(ascending=False)
axes[1,0].bar(touchpoint_lift.index, touchpoint_lift.values)
axes[1,0].set_title('Average KPI Lift by Touchpoint')
axes[1,0].set_xlabel('Touchpoint')
axes[1,0].set_ylabel('Average KPI Lift')

# KPI ROI by Touchpoint
touchpoint_roi = promos_df.groupby('touchpoint')['kpi_roi'].mean().sort_values(ascending=False)
axes[1,1].bar(touchpoint_roi.index, touchpoint_roi.values)
axes[1,1].set_title('Average KPI ROI by Touchpoint')
axes[1,1].set_xlabel('Touchpoint')
axes[1,1].set_ylabel('Average KPI ROI')

plt.tight_layout()
plt.show()

print("KPI Performance Summary:")
print(f"KPI Lift - Mean: {promos_df['kpi_lift'].mean():.3f}, Std: {promos_df['kpi_lift'].std():.3f}")
print(f"KPI ROI - Mean: {promos_df['kpi_roi'].mean():.3f}, Std: {promos_df['kpi_roi'].std():.3f}")
print("\nKPI Lift by Touchpoint:")
print(touchpoint_lift)
print("\nKPI ROI by Touchpoint:")
print(touchpoint_roi)

## 6. Performance by Category and Brand

In [ ]:
# Performance analysis by category and brand
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# KPI by Category
category_performance = promos_df.groupby('category')[['kpi_lift', 'kpi_roi']].mean()
category_performance.plot(kind='bar', ax=axes[0,0])
axes[0,0].set_title('Average KPI Performance by Category')
axes[0,0].set_xlabel('Category')
axes[0,0].set_ylabel('Average KPI')
axes[0,0].legend()
axes[0,0].tick_params(axis='x', rotation=45)

# KPI by Brand
brand_performance = promos_df.groupby('brand')[['kpi_lift', 'kpi_roi']].mean().sort_values('kpi_roi', ascending=False)
brand_performance.plot(kind='bar', ax=axes[0,1])
axes[0,1].set_title('Average KPI Performance by Brand')
axes[0,1].set_xlabel('Brand')
axes[0,1].set_ylabel('Average KPI')
axes[0,1].legend()
axes[0,1].tick_params(axis='x', rotation=45)

# Scatter plot: KPI Lift vs ROI
scatter = axes[1,0].scatter(promos_df['kpi_lift'], promos_df['kpi_roi'], 
                          c=promos_df['touchpoint'].astype('category').cat.codes, 
                          alpha=0.7, s=50)
axes[1,0].set_title('KPI Lift vs ROI by Touchpoint')
axes[1,0].set_xlabel('KPI Lift')
axes[1,0].set_ylabel('KPI ROI')

# Create a colorbar legend
touchpoints = promos_df['touchpoint'].unique()
for i, touchpoint in enumerate(touchpoints):
    axes[1,0].scatter([], [], c=f'C{i}', label=touchpoint)
axes[1,0].legend(title='Touchpoint', bbox_to_anchor=(1.05, 1), loc='upper left')

# Performance distribution by period
period_performance = promos_df.groupby('period')[['kpi_lift', 'kpi_roi']].mean()
period_performance.plot(kind='bar', ax=axes[1,1])
axes[1,1].set_title('Average KPI Performance by Period')
axes[1,1].set_xlabel('Period')
axes[1,1].set_ylabel('Average KPI')
axes[1,1].legend()
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("Performance by Category:")
print(category_performance)
print("\nPerformance by Brand:")
print(brand_performance)
print("\nPerformance by Period:")
print(period_performance)

## 7. Promotion Text Analysis

In [ ]:
# Text analysis of promotion headlines
import re
from collections import Counter

# Extract key terms from headlines
all_headlines = ' '.join(promos_df['headline'].str.lower())
words = re.findall(r'\b\w+\b', all_headlines)
word_freq = Counter(words)

# Remove common words
stop_words = {'dan', 'atau', 'dari', 'untuk', 'dengan', 'ke', 'di', 'pada', 'yang', 'adalah', 'beli', 'rp'}
filtered_words = {word: freq for word, freq in word_freq.items() if word not in stop_words and len(word) > 2}

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Most common words in headlines
top_words = dict(sorted(filtered_words.items(), key=lambda x: x[1], reverse=True)[:15])
axes[0,0].bar(top_words.keys(), top_words.values())
axes[0,0].set_title('Most Common Words in Promotion Headlines')
axes[0,0].set_xlabel('Words')
axes[0,0].set_ylabel('Frequency')
axes[0,0].tick_params(axis='x', rotation=45)

# Word cloud
if len(filtered_words) > 0:
    wordcloud = WordCloud(width=400, height=300, background_color='white').generate_from_frequencies(filtered_words)
    axes[0,1].imshow(wordcloud, interpolation='bilinear')
    axes[0,1].set_title('Word Cloud of Promotion Headlines')
    axes[0,1].axis('off')

# Headline length analysis
promos_df['headline_length'] = promos_df['headline'].str.len()
axes[1,0].hist(promos_df['headline_length'], bins=15, edgecolor='black', alpha=0.7)
axes[1,0].set_title('Distribution of Headline Lengths')
axes[1,0].set_xlabel('Headline Length (characters)')
axes[1,0].set_ylabel('Frequency')
axes[1,0].axvline(promos_df['headline_length'].mean(), color='red', linestyle='--', 
                  label=f'Mean: {promos_df["headline_length"].mean():.1f}')
axes[1,0].legend()

# Description length analysis
promos_df['description_length'] = promos_df['description'].str.len()
axes[1,1].hist(promos_df['description_length'], bins=15, edgecolor='black', alpha=0.7)
axes[1,1].set_title('Distribution of Description Lengths')
axes[1,1].set_xlabel('Description Length (characters)')
axes[1,1].set_ylabel('Frequency')
axes[1,1].axvline(promos_df['description_length'].mean(), color='red', linestyle='--', 
                  label=f'Mean: {promos_df["description_length"].mean():.1f}')
axes[1,1].legend()

plt.tight_layout()
plt.show()

print("Text Analysis Summary:")
print(f"Average headline length: {promos_df['headline_length'].mean():.1f} characters")
print(f"Average description length: {promos_df['description_length'].mean():.1f} characters")
print(f"\nTop 10 words in headlines: {dict(list(top_words.items())[:10])}")

## 8. Correlation Analysis

In [ ]:
# Correlation analysis
numerical_cols = ['kpi_lift', 'kpi_roi', 'headline_length', 'description_length']
correlation_matrix = promos_df[numerical_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=0.5)
plt.title('Correlation Matrix of Numerical Variables')
plt.tight_layout()
plt.show()

print("Correlation Matrix:")
print(correlation_matrix)

# Statistical summary
print("\n=== Statistical Summary ===")
print(promos_df[numerical_cols].describe())

## 9. Performance Insights by Promotion Type

In [ ]:
# Extract promotion types from headlines
def extract_promo_type(headline):
    headline = headline.lower()
    if 'diskon' in headline or '%' in headline:
        return 'Discount'
    elif 'gratis' in headline or 'free' in headline:
        return 'Free/Gift'
    elif 'beli' in headline and ('gratis' in headline or 'dapat' in headline):
        return 'Buy X Get Y'
    elif 'hemat' in headline:
        return 'Save Money'
    elif 'win' in headline or 'menang' in headline or 'kirim' in headline:
        return 'Contest/Game'
    elif 'sampling' in headline or 'coba' in headline:
        return 'Sampling'
    else:
        return 'Other'

promos_df['promo_type'] = promos_df['headline'].apply(extract_promo_type)

# Analysis by promotion type
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Distribution of promotion types
promo_type_counts = promos_df['promo_type'].value_counts()
axes[0,0].pie(promo_type_counts.values, labels=promo_type_counts.index, autopct='%1.1f%%')
axes[0,0].set_title('Distribution of Promotion Types')

# Performance by promotion type
promo_type_performance = promos_df.groupby('promo_type')[['kpi_lift', 'kpi_roi']].mean().sort_values('kpi_roi', ascending=False)
promo_type_performance.plot(kind='bar', ax=axes[0,1])
axes[0,1].set_title('Average KPI Performance by Promotion Type')
axes[0,1].set_xlabel('Promotion Type')
axes[0,1].set_ylabel('Average KPI')
axes[0,1].legend()
axes[0,1].tick_params(axis='x', rotation=45)

# Box plot: KPI Lift by Promotion Type
promos_df.boxplot(column='kpi_lift', by='promo_type', ax=axes[1,0])
axes[1,0].set_title('KPI Lift Distribution by Promotion Type')
axes[1,0].set_xlabel('Promotion Type')
axes[1,0].set_ylabel('KPI Lift')
axes[1,0].tick_params(axis='x', rotation=45)

# Box plot: KPI ROI by Promotion Type
promos_df.boxplot(column='kpi_roi', by='promo_type', ax=axes[1,1])
axes[1,1].set_title('KPI ROI Distribution by Promotion Type')
axes[1,1].set_xlabel('Promotion Type')
axes[1,1].set_ylabel('KPI ROI')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("Promotion Type Analysis:")
print("Distribution:")
print(promo_type_counts)
print("\nPerformance by Type:")
print(promo_type_performance)

## 10. Key Findings and Recommendations

In [ ]:
# Summary statistics and insights
print("=== KEY FINDINGS AND INSIGHTS ===")
print("\n1. DATASET OVERVIEW:")
print(f"   - SKU Master: {sku_df.shape[0]} products across {sku_df['brand'].nunique()} brands")
print(f"   - Promotions: {promos_df.shape[0]} promotions across {promos_df['period'].nunique()} periods")
print(f"   - Categories: {promos_df['category'].nunique()} categories represented")

print("\n2. PERFORMANCE INSIGHTS:")
best_touchpoint_lift = touchpoint_lift.index[0]
best_touchpoint_roi = touchpoint_roi.index[0]
print(f"   - Best performing touchpoint (Lift): {best_touchpoint_lift} ({touchpoint_lift.iloc[0]:.3f})")
print(f"   - Best performing touchpoint (ROI): {best_touchpoint_roi} ({touchpoint_roi.iloc[0]:.3f})")

best_category = category_performance.sort_values('kpi_roi', ascending=False).index[0]
print(f"   - Best performing category: {best_category}")

best_promo_type = promo_type_performance.index[0]
print(f"   - Best performing promotion type: {best_promo_type}")

print("\n3. TEXT ANALYSIS:")
print(f"   - Average headline length: {promos_df['headline_length'].mean():.1f} characters")
print(f"   - Most common promotional words: {list(top_words.keys())[:5]}")

print("\n4. DATA QUALITY:")
print(f"   - SKU data completeness: {((sku_df.shape[0] * sku_df.shape[1] - sku_df.isnull().sum().sum()) / (sku_df.shape[0] * sku_df.shape[1]) * 100):.1f}%")
print(f"   - Promotion data completeness: {((promos_df.shape[0] * promos_df.shape[1] - promos_df.isnull().sum().sum()) / (promos_df.shape[0] * promos_df.shape[1]) * 100):.1f}%")

print("\n5. RECOMMENDATIONS FOR COPY RE-USE SYSTEM:")
print("   - Focus on high-performing touchpoints and promotion types")
print("   - Consider semantic similarity based on headline structure and key terms")
print("   - Weight recommendations by historical KPI performance")
print("   - Account for category and brand context in matching")
print("   - Use text length as a filtering criterion for similar contexts")

## 11. Data Export for Further Analysis

In [ ]:
# Create enhanced dataset with derived features
enhanced_promos = promos_df.copy()
enhanced_promos['promo_type'] = promos_df['promo_type']
enhanced_promos['headline_length'] = promos_df['headline_length']
enhanced_promos['description_length'] = promos_df['description_length']

# Performance quartiles
enhanced_promos['kpi_lift_quartile'] = pd.qcut(enhanced_promos['kpi_lift'], 4, labels=['Low', 'Medium-Low', 'Medium-High', 'High'])
enhanced_promos['kpi_roi_quartile'] = pd.qcut(enhanced_promos['kpi_roi'], 4, labels=['Low', 'Medium-Low', 'Medium-High', 'High'])

print("Enhanced dataset created with the following new features:")
print("- promo_type: Extracted promotion category")
print("- headline_length: Character count of headlines")
print("- description_length: Character count of descriptions")
print("- kpi_lift_quartile: Performance quartile for lift")
print("- kpi_roi_quartile: Performance quartile for ROI")

print(f"\nEnhanced dataset shape: {enhanced_promos.shape}")
print("\nFirst 3 rows of enhanced dataset:")
print(enhanced_promos.head(3))

# Save enhanced dataset
enhanced_promos.to_csv('data/enhanced_promos.csv', index=False)
print("\nEnhanced dataset saved as 'data/enhanced_promos.csv'")